In [1]:
# Example of a fuzzy match across multiple columns, with blocking.

In [2]:
import numpy as np
import numbers
import pandas as pd

import phonetics

from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

In [3]:
import IPython

In [4]:
import vega

Setup
==

Vega setup for charts
--
Splink uses Vega for inline charts. We need a bit of setup to make sure these are displayed correctly.

In [5]:
# !jupyter nbextension install --sys-prefix --py vega

In [6]:
!jupyter nbextension enable vega --py --sys-prefix

Enabling notebook extension jupyter-vega/extension...
      - Validating: ok


In [7]:
# # A minimal example for debugging:
# vega.Vega({
#     "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
#     "data": {
#         "values": [
#             {"a": "A", "b": 28},
#             {"a": "B", "b": 55},
#             {"a": "C", "b": 43},
#         ]
#     },
#     "mark": "bar",
#     "encoding": {
#         "x": {"field": "b", "type": "quantitative"},
#         "y": {"field": "a", "type": "nominal"}
#     }
# })

Data
==

Tables
--

In [8]:
table1 = pd.read_csv('data/interim/table1.csv', dtype=str)
table1.head(2)

,unique_id,city,postcode,full_name,first_and_surname,first_name,surname,dob,gender,cluster
0,Q2296770-2,devon,tq13 8df,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,male,Q2296770
1,Q1443188-1,bristol,bs2 0el,frank edward brightman,frank brightman,frank,brightman,1856-06-18,male,Q1443188


In [9]:
table2 = pd.read_csv('data/interim/table2.csv', dtype=str)
table2.head(2)

,unique_id,city,postcode,full_name,first_and_surname,first_name,surname,dob,gender,cluster
0,Q2296770-4,devon,tq13 8hu,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,NaN,Q2296770
1,Q1443188-2,bristol,bs2 0el,frank edward brightman,frank brightman,frank,brightman,1856-06-18,male,Q1443188


Fuzzy match
==
Based on https://moj-analytical-services.github.io/splink/demos/examples/duckdb/link_only.html

Linking two tables without deduplication.

Settings
--

In [10]:
person_match_settings = {
    "link_type": "link_only",
    "unique_id_column_name": "unique_id",
    
    # Blocking, for performance
    "blocking_rules_to_generate_predictions": [
        "l.postcode = r.postcode",
    ],
    "comparisons": [
        cl.exact_match("city", term_frequency_adjustments=False),
        
        # Probabilistic name match (string distances)
        
        # For documentation of the name comparators used here, see:
        # https://moj-analytical-services.github.io/splink/comparison_template_library.html
        
        # For advice on choosing good thresholds see: 
        # https://moj-analytical-services.github.io/splink/topic_guides/comparisons/choosing_comparators.html
        
        ctl.name_comparison("first_name"), 
#         ctl.name_comparison("first_name", 
#                             term_frequency_adjustments=False,
#                             levenshtein_thresholds=[1],         # No. of single-char insert/delete/subst
#                             damerau_levenshtein_thresholds=[],  # ... or 2-char transpose
#                             jaro_thresholds=[],                 # Character occurrences regardless of order
#                             jaro_winkler_thresholds=[],         # ... with preference for beginning of string
#                             jaccard_thresholds=[]),
        ctl.name_comparison("surname"), 
#         ctl.name_comparison("full_name"), 

#         ctl.forename_surname_comparison("first_name", "surname"),
#         ctl.forename_surname_comparison("first_name", "surname",
#                                         term_frequency_adjustments=False,
#                                         include_columns_reversed=False,
#                                         levenshtein_thresholds=[],
#                                         damerau_levenshtein_thresholds=[], 
#                                         jaro_thresholds=[], 
#                                         jaro_winkler_thresholds=[0.88], 
#                                         jaccard_thresholds=[]),
    ],
    
    # Needed for waterfall charts of results
    "retain_intermediate_calculation_columns": True,
    "retain_matching_columns": True,
}

Training
--

In [11]:
person_linker = DuckDBLinker(
    [table1, table2],
    person_match_settings)

In [12]:
person_linker.missingness_chart()

alt.LayerChart(...)

In [13]:
person_linker.estimate_probability_two_random_records_match(
    #  A list of deterministic matching rules that should be designed to admit 
    # very few (none if possible) false positives:
    [
        "l.city = r.city and l.full_name = r.full_name",
    ], 
    recall=0.9)

Probability two random records match is estimated to be  7.9e-05.
This means that amongst all possible pairwise record comparisons, one in 12,652.24 are expected to match.  With 19,386,048 total possible comparisons, we expect a total of around 1,532.22 matching pairs


In [14]:
# Estimate U
person_linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - city (no m values are trained).
    - first_name (no m values are trained).
    - surname (no m values are trained).


In [15]:
session_person_uid = person_linker.estimate_parameters_using_expectation_maximisation(
    "l.city = r.city")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.city = r.city

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - city


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Iteration 1: Largest change in params was -0.11 in the m_probability of first_name, level `Exact match first_name`
Iteration 2: Largest change in params was -0.033 in the m_probability of first_name, level `Exact match first_name`
Iteration 3: Largest change in params was 0.0244 in the m_probability of first_name, level `All other comparisons`
Iteration 4: Largest change in params was 0.0165 in the m_probability of first_name, level `All other comparisons`
Iteration 5: Largest change in params was 0.0079 in the m_probability of first_name, level `All other comparisons`
Iteration 6: Largest change in params was 0.00324 in the m_probability of first_name, level `All other comparisons`
Iteration 7: Largest change in params was 0.00126 in the m_probability of first_name, level `All other comparisons`
Iteration 8: Largest change in params was 0.000475 in the m_probability of first_name, level `All other comparisons`
Iteration 9: Largest change in params was 0.000179 in the m_probability of

In [16]:
session_person_fsn = person_linker.estimate_parameters_using_expectation_maximisation(
    "l.first_name = r.first_name and l.surname = r.surname")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - city

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Iteration 1: Largest change in params was -0.328 in probability_two_random_records_match
Iteration 2: Largest change in params was -0.0165 in the m_probability of city, level `Exact match`
Iteration 3: Largest change in params was 0.000963 in the m_probability of city, level `All other comparisons`
Iteration 4: Largest change in params was 5.46e-05 in the m_probability of city, level `All other comparisons`

EM converged after 4 iterations

Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [17]:
session_person_fulln = person_linker.estimate_parameters_using_expectation_maximisation(
    "l.full_name  = r.full_name ")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.full_name  = r.full_name 

Parameter estimates will be made for the following comparison(s):
    - city
    - first_name
    - surname

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 

Level Damerau_levenshtein <= 1 on comparison first_name not observed in dataset, unable to train m value

Level Jaro_winkler_similarity >= 0.9 on comparison first_name not observed in dataset, unable to train m value

Level Jaro_winkler_similarity >= 0.8 on comparison first_name not observed in dataset, unable to train m value

Level All other comparisons on comparison first_name not observed in dataset, unable to train m value

Level Damerau_levenshtein <= 1 on comparison surname not observed in dataset, unable to train m value

Level Jaro_winkler_similarity >= 0.9 on comparison surname not observed in dataset, unable to train m value

Le

Review the parameters
--

In [18]:
person_linker.match_weights_chart()

alt.VConcatChart(...)

In [19]:
person_linker.parameter_estimate_comparisons_chart()

alt.Chart(...)

In [20]:
person_linker.unlinkables_chart()

alt.LayerChart(...)

Linking
--

In [21]:
# Match probability threshold
# See the unlinkables chart above to determine a good value for this threshold, 
# and the match probabilities below for added context
person_match_threshold = 0.99

person_match_results = person_linker.predict(threshold_match_probability=person_match_threshold) 
person_match = person_match_results.as_pandas_dataframe()
person_match.sample(10)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,city_l,city_r,gamma_city,bf_city,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,gamma_surname,bf_surname,postcode_l,postcode_r
713,7.625249,0.994961,__splink__input_table_0,__splink__input_table_1,Q28037657-4,Q28037657-3,colchester,colchester,1,136.108258,charles,carl,1,18.186183,marshall,marshall,4,1009.103537,co4 5hp,co4 5hp
273,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q3401572-2,Q3401572-1,plas penmynydd,plas penmynydd,1,136.108258,robert,robert,4,45.022541,huw,huw,4,1009.103537,sa19 9uw,sa19 9uw
514,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q6139106-2,Q6139106-1,scarborough,scarborough,1,136.108258,james,james,4,45.022541,wilson,wilson,4,1009.103537,yo14 9bl,yo14 9bl
703,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q21453621-1,Q21453621-2,cherwell,cherwell,1,136.108258,henry,henry,4,45.022541,dawson,dawson,4,1009.103537,ox16 3jj,ox16 3jj
449,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q5536979-1,Q5536979-2,blackpool,blackpool,1,136.108258,george,george,4,45.022541,bigg,bigg,4,1009.103537,fy4 4nj,fy4 4nj
1419,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q2741203-2,Q2741203-1,east saltoun and west saltoun,east saltoun and west saltoun,1,136.108258,john,john,4,45.022541,thomson,thomson,4,1009.103537,eh34 5eb,eh34 5eb
125,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q711325-1,Q711325-2,leicester,leicester,1,136.108258,arthur,arthur,4,45.022541,newman,newman,4,1009.103537,le5 3gg,le5 3gg
1371,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q4529867-1,Q4529867-2,london,london,1,136.108258,julian,julian,4,45.022541,egerton,egerton,4,1009.103537,se11 6qu,se11 6qu
1527,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q76847659-2,Q76847659-1,exeter,exeter,1,136.108258,william,william,4,45.022541,lloyd,lloyd,4,1009.103537,ex3 0an,ex3 0an
828,8.933053,0.997958,__splink__input_table_0,__splink__input_table_1,Q55720024-1,Q55720024-2,walsall,walsall,1,136.108258,susan,susan,4,45.022541,mcbeth,mcbeth,4,1009.103537,ws3 5ey,ws3 5ey


In [22]:
# Exact match = 1370
len(person_match)

1585

In [23]:
person_match.sort_values(by='match_probability', ascending=True).head(15)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,city_l,city_r,gamma_city,bf_city,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,gamma_surname,bf_surname,postcode_l,postcode_r
1006,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q4821397-11,Q4821397-1,dover,dover,1,136.108258,augustus,augustus,4,45.022541,clarke,clark,3,209.674524,ct15 5lp,ct15 5lp
1209,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q7817912-9,Q7817912-1,scotland,scotland,1,136.108258,tom,tom,4,45.022541,vallsnce,vallance,3,209.674524,ub8 3tn,ub8 3tn
393,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q26722118-3,Q26722118-5,st. helens,st. helens,1,136.108258,justinian,justinian,4,45.022541,edwards-geathcote,edwards-heathcote,3,209.674524,wa9 2by,wa9 2by
37,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q5236742-2,Q5236742-1,gdańsk,gdańsk,1,136.108258,david,david,4,45.022541,logban,loggan,3,209.674524,s71 2hu,s71 2hu
38,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q19585833-1,Q19585833-2,mendip,mendip,1,136.108258,john,john,4,45.022541,keppie,kep0ie,3,209.674524,ba4 5jt,ba4 5jt
570,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q14623866-4,Q14623866-3,south norfolk,south norfolk,1,136.108258,richmond,richmond,4,45.022541,abbot,abbott,3,209.674524,nr14 6jb,nr14 6jb
446,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q12353681-2,Q12353681-5,lewisham,lewisham,1,136.108258,paul,paul,4,45.022541,mathews,matthews,3,209.674524,se14 6je,se14 6je
948,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q65665593-1,Q65665593-4,lewes,lewes,1,136.108258,charles,charles,4,45.022541,exon,edon,3,209.674524,bn10 7bd,bn10 7bd
952,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q5414294-7,Q5414294-5,exeter,exeter,1,136.108258,eustace,eustace,4,45.022541,hudgell,budgell,3,209.674524,ex4 4pw,ex4 4pw
738,6.666203,0.99025,__splink__input_table_0,__splink__input_table_1,Q6929970-1,Q6929970-4,cotswold,cotswold,1,136.108258,mrs,mrs,4,45.022541,philarmonica,philarmonifa,3,209.674524,gl7 1tt,gl7 1tt


In [24]:
person_match.match_weight.value_counts()

match_weight
8.933053    1352
7.625249      85
6.666203      60
8.312912      60
6.699013      28
Name: count, dtype: int64

In [25]:
person_match.match_probability.value_counts()

match_probability
0.997958    1352
0.994961      85
0.990250      60
0.996865      60
0.990467      28
Name: count, dtype: int64

Inspect the results
--

In [26]:
person_linker.waterfall_chart(person_match_results.as_record_dict(limit=20))

alt.LayerChart(...)

In [27]:
# Comparison viewer
person_linker.comparison_viewer_dashboard(person_match_results, 
                                          "data/interim/person_match-fuzzy-report-scv.html", 
                                          overwrite=True, num_example_rows=20)
IPython.display.IFrame(src="data/interim/person_match-fuzzy-report-scv.html", width="100%", height=1200)

Export
--

In [28]:
person_match.to_csv('data/processed/person_match-fuzzy.csv', index=False)